### Loading the models and creating masks using Activation maps

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
# For activation maps
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [3]:
from torchvision.models import resnet50
from torch import nn

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
path_images = "D:\\Study\\Conda Projects\\opdl\\Data\\images\\images"

#### Oxford Masks Creation

In [7]:
path_masks_oxford_resnet50= "D:\\Study\\Conda Projects\\opdl\\Data\\oxford\\resnet50"

In [8]:
model_state_path = "D:\\Study\\Conda Projects\\opdl\\checkpoints\\model_oxf_resnet50_epochs10.pth"

In [9]:
model = resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features,1)
model.to(device)
model.load_state_dict(torch.load(model_state_path), strict=False)

d:\Study\Conda Projects\opdl\opdl\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Study\Conda Projects\opdl\opdl\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [10]:
target_layers = [model.layer4[-1]]

In [23]:
activationMethod = {'gradcam':GradCAM(model=model, target_layers=target_layers),
                    'hirescam': HiResCAM(model=model, target_layers=target_layers),
                    'fullgrad':FullGrad(model=model, target_layers=target_layers)}

In [12]:
# Defining the transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224))
])

In [13]:
def create_save_masks(img_path,cam):
    test_image = cv2.imread(img_path)
    test_image = transform(test_image)
    test_image = test_image.unsqueeze(0)
    test_image = test_image.to(device)
    grayscale_cam = cam(input_tensor=test_image)
    grayscale_cam = grayscale_cam[0, :]
    grayscale_cam_img = np.where(grayscale_cam>0.58,255,0)
    return grayscale_cam_img

In [19]:
def list_files_in_directory(directory,save_path,cam):
    save_path += "_"+str(cam)
    cam = activationMethod[cam]
    cnt=0
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                cnt+=1
                mask = create_save_masks(os.path.join(directory,filename),cam)
                cv2.imwrite(os.path.join(save_path,filename.split(".")[0]+'-mask.jpg'), mask)
    return "Processed "+str(cnt)+" images"

In [24]:
list_files_in_directory(path_images,path_masks_oxford_resnet50,'fullgrad')

'Processed 7390 images'